In [334]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import os
import re
import spacy
import warnings
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
warnings.filterwarnings('ignore')

In [335]:
df = pd.read_csv('train.csv')
dft = pd.read_csv("test.csv")
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [336]:
print(df.isna().sum())
df.drop(["Name"],axis=1,inplace=True)
dft.drop(["Name"],axis=1,inplace=True)
for col in df.columns:
  df[col] = df[col].fillna(df[col].mode()[0])
for col in dft.columns:
  dft[col] = dft[col].fillna(dft[col].mode()[0])
print(df.isna().sum())

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64
PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64


In [337]:
dir(df)

['Age',
 'Cabin',
 'CryoSleep',
 'Destination',
 'FoodCourt',
 'HomePlanet',
 'PassengerId',
 'RoomService',
 'ShoppingMall',
 'Spa',
 'T',
 'Transported',
 'VIP',
 'VRDeck',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__arrow_c_stream__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__dataframe__',
 '__dataframe_consortium_standard__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',


In [338]:
df["HomePlanet"].value_counts()

,count
HomePlanet,
Earth,4803
Europa,2131
Mars,1759


In [339]:
df[['CryoSleep', 'VIP', 'Transported']] = df[['CryoSleep', 'VIP', 'Transported']].astype(int)
df[['CabinDeck', 'CabinNum', 'CabinSide']] = df['Cabin'].str.split('/', expand=True)
dft[['CryoSleep', 'VIP']] = dft[['CryoSleep', 'VIP']].astype(int)
dft[['CabinDeck', 'CabinNum', 'CabinSide']] = dft['Cabin'].str.split('/', expand=True)
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,CabinDeck,CabinNum,CabinSide
0,0001_01,Europa,0,B/0/P,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,0,B,0,P
1,0002_01,Earth,0,F/0/S,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,1,F,0,S
2,0003_01,Europa,0,A/0/S,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,A,0,S
3,0003_02,Europa,0,A/0/S,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,A,0,S
4,0004_01,Earth,0,F/1/S,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,1,F,1,S


In [340]:
cat_cols = ['HomePlanet', 'Destination','CabinDeck', 'CabinSide']

ct = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ],
    remainder='passthrough'
)

df1 = ct.fit_transform(df)
dft1 = ct.fit_transform(dft)
df1

array([[0.0, 1.0, 0.0, ..., 0.0, 0, '0'],
       [1.0, 0.0, 0.0, ..., 44.0, 1, '0'],
       [0.0, 1.0, 0.0, ..., 49.0, 0, '0'],
       ...,
       [1.0, 0.0, 0.0, ..., 0.0, 1, '1500'],
       [0.0, 1.0, 0.0, ..., 3235.0, 0, '608'],
       [0.0, 1.0, 0.0, ..., 12.0, 1, '608']], dtype=object)

In [341]:
ohe_feature_names = ct.named_transformers_['cat'].get_feature_names_out(cat_cols)
other_cols = [col for col in df.columns if col not in cat_cols]
all_cols = list(ohe_feature_names) + other_cols
df = pd.DataFrame(df1, columns=all_cols)
other_cols = [col for col in dft.columns if col not in cat_cols]
all_cols = list(ohe_feature_names) + other_cols
dft = pd.DataFrame(dft1, columns=all_cols)
df.head()

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,CabinDeck_A,CabinDeck_B,CabinDeck_C,CabinDeck_D,...,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,CabinNum
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,B/0/P,39.0,0,0.0,0.0,0.0,0.0,0.0,0,0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,F/0/S,24.0,0,109.0,9.0,25.0,549.0,44.0,1,0
2,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,A/0/S,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,0
3,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,A/0/S,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,F/1/S,16.0,0,303.0,70.0,151.0,565.0,2.0,1,1


In [342]:
std = StandardScaler()
for col in ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'CabinNum']:
    df[col] = std.fit_transform(df[[col]])
    dft[col] = std.fit_transform(dft[[col]])
dft.head()

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,CabinDeck_A,CabinDeck_B,CabinDeck_C,CabinDeck_D,...,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,CabinNum
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1,G/3/S,-0.101445,0,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,-1.162210
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,F/4/S,-0.668426,0,-0.357339,-0.277879,-0.312173,2.287504,-0.246712,-1.160262
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1,C/0/S,0.182045,0,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,-1.168053
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0,C/1/S,0.678153,0,-0.357339,4.121518,-0.312173,-0.104002,0.226648,-1.166105
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,F/5/S,-0.597554,0,-0.340723,-0.283840,0.832122,-0.267841,-0.246712,-1.158314


In [343]:
df['GroupID'] = df['PassengerId'].apply(lambda x: x.split('_')[0])
group_sizes = df['GroupID'].value_counts()
df['GroupSize'] = df['GroupID'].map(group_sizes)
df['IsSolo'] = df['GroupSize'].apply(lambda x: 1 if x == 1 else 0)
df.drop(["Cabin","PassengerId", "GroupSize", "GroupID","HomePlanet_Europa","Destination_TRAPPIST-1e","CabinDeck_A","CabinSide_P"], axis=1, inplace=True)
x_train = df.drop("Transported", axis=1)
y_train = df["Transported"].astype(int)
dft['GroupID'] = dft['PassengerId'].apply(lambda x: x.split('_')[0])
group_sizes = dft['GroupID'].value_counts()
dft['GroupSize'] = dft['GroupID'].map(group_sizes)
dft['IsSolo'] = dft['GroupSize'].apply(lambda x: 1 if x == 1 else 0)
dft.drop(["Cabin","PassengerId", "GroupSize", "GroupID","HomePlanet_Europa","Destination_TRAPPIST-1e","CabinDeck_A","CabinSide_P"], axis=1, inplace=True)
x_test = dft
dft.head()

,HomePlanet_Earth,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,CabinDeck_B,CabinDeck_C,CabinDeck_D,CabinDeck_E,CabinDeck_F,CabinDeck_G,...,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,CabinNum,IsSolo
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1,-0.101445,0,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,-1.162210,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,-0.668426,0,-0.357339,-0.277879,-0.312173,2.287504,-0.246712,-1.160262,1
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1,0.182045,0,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,-1.168053,1
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0.678153,0,-0.357339,4.121518,-0.312173,-0.104002,0.226648,-1.166105,1
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,-0.597554,0,-0.340723,-0.283840,0.832122,-0.267841,-0.246712,-1.158314,1


In [344]:
df = df.astype(float)
dft = dft.astype(float)
x_train = x_train.astype(float)
y_train = y_train.astype(float)
x_test = x_test.astype(float)
print(df.dtypes)

HomePlanet_Earth             float64
HomePlanet_Mars              float64
Destination_55 Cancri e      float64
Destination_PSO J318.5-22    float64
CabinDeck_B                  float64
CabinDeck_C                  float64
CabinDeck_D                  float64
CabinDeck_E                  float64
CabinDeck_F                  float64
CabinDeck_G                  float64
CabinDeck_T                  float64
CabinSide_S                  float64
CryoSleep                    float64
Age                          float64
VIP                          float64
RoomService                  float64
FoodCourt                    float64
ShoppingMall                 float64
Spa                          float64
VRDeck                       float64
Transported                  float64
CabinNum                     float64
IsSolo                       float64
dtype: object


In [345]:
log_clf = LogisticRegression(max_iter=1000)
rf_clf = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

voting_clf = VotingClassifier(
    estimators=[
        ('log', log_clf),
        ('rf', rf_clf),
        ('xgb', xgb_clf)
    ],
    voting='soft'
)

pipeline = Pipeline([
    ('model', voting_clf)
])

pipeline.fit(x_train, y_train)


Pipeline(steps=[('model',
                 VotingClassifier(estimators=[('log',
                                               LogisticRegression(max_iter=1000)),
                                              ('rf',
                                               RandomForestClassifier(max_depth=10,
                                                                      n_estimators=200,
                                                                      random_state=42)),
                                              ('xgb',
                                               XGBClassifier(base_score=None,
                                                             booster=None,
                                                             callbacks=None,
                                                             colsample_bylevel=None,
                                                             colsample_bynode=None,
                                                             colsample_bytree=None,
                                                             device=None,
                                                             early_stopping_rounds=None,
                                                             enable...
                                                             importance_type=None,
                                                             interaction_constraints=None,
                                                             learning_rate=None,
                                                             max_bin=None,
                                                             max_cat_threshold=None,
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=None,
                                                             max_leaves=None,
                                                             min_child_weight=None,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=None,
                                                             n_jobs=None,
                                                             num_parallel_tree=None,
                                                             random_state=42, ...))],
                                  voting='soft'))])

In [346]:
preds = pipeline.predict(x_test)
submission = pd.read_csv("sample_submission.csv")
submission['Transported'] = preds.astype(bool)
submission.to_csv("submission.csv", index=False)